In [144]:
import numpy as np
import pandas as pd
import emoji

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Embedding,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [145]:
data = pd.read_csv('emoji_data.csv',header = None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [146]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize((emoji_dict[label]))
    

In [147]:
X = data[0].values
Y = data[1].values

In [148]:
file = open('glove.6B.100d.txt','r',encoding = 'utf8')
content = file.readlines()
file.close()

In [149]:
embeddings = {}
for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:],dtype = float)

In [150]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
wordToIndex = tokenizer.word_index

In [151]:
# All sentences are of different length so do padding to make them of same length
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen,len(sent))
    return maxlen
maxlen = get_maxlen(Xtokens)


In [152]:
Xtokens = tokenizer.texts_to_sequences(X)
Xtrain = pad_sequences(Xtokens,maxlen = maxlen,padding = 'post',truncating='post')

In [153]:
# Ytrain = to_categorical(Y)
Ytrain = to_categorical(Y)

In [154]:
embed_size = 100
embedding_matrix = np.zeros((len(wordToIndex)+1,embed_size))
for word,i in wordToIndex.items():
    embed_vector = embeddings[word]
    embedding_matrix[i] = embed_vector

In [155]:

model = Sequential()
embedding_layer = Embedding(input_dim = len(wordToIndex)+1,
          output_dim = embed_size,
          input_shape = (maxlen,)
         )
model.add(embedding_layer)
model.add(LSTM(units = 16,return_sequences = True))
model.add(Dropout(0.5))
model.add(LSTM(units = 4))
model.add(Dropout(0.5))

model.add(Dense(5,activation='softmax'))

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.summary()

C:\Users\MuhammadAhmadBajwa\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ (None, 10, 100)             │          31,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 10, 16)              │           7,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 10, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 4)                   │             336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 4)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │              25 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 39,149 (152.93 KB)

 Trainable params: 7,849 (30.66 KB)

 Non-trainable params: 31,300 (122.27 KB)

In [156]:
model.fit(Xtrain,Ytrain,epochs=100)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.1872 - loss: 1.6076
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1590 - loss: 1.5978 
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2482 - loss: 1.5931 
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2608 - loss: 1.5996 
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1773 - loss: 1.5818 
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2826 - loss: 1.5786 
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3054 - loss: 1.5710 
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3219 - loss: 1.5298 
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3106 - loss: 1.5177
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2987 - loss: 1.4971 
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3054 - loss: 1.5065 
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4132 - los

In [157]:
test = ["I feel good", "I feel very bad", "I am Hungry"]
test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq,maxlen=maxlen,padding='post',truncating='post')
y_predict = model.predict(Xtest)
y_predict = np.argmax(y_predict,axis=1)

for i in range(len(test)):
    print(test[i],label_to_emoji(y_predict[i]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step
I feel good 😃
I feel very bad 😞
I am Hungry 🍽️
